## План занятия
- [Случайные величины. Нормальный закон распределения.](#rv)
    - [Распределение вероятностей](#pd)
    - [ Функция распределения](#cdf)
    - [Нормальный закон распределени](#nb)
        - [Закон больших чисел и центральная предельная теорема](#lbn)
        - [Формулы для функции и плотности нормального распределения](#pdf)
        - [ Генерирование выборки, графики функций распределения и плотностей, эмпирических и теоретических](#gr)
    - [Нахождение доверительных интервалов](#ci)
    - [Задание](#t1)
    
- [ A/B тестирование](#ab)
    - [Первичный анализ данных](#eda)
    - [ Статистический вывод для A/B теста](#si)
        - [Формулировка рабочей гипотезы](#si1)
        - [Формальное описание нулевой и альтернативной гипотез](#si2)
        - [ Выбор подходящего статистического теста (статистического критерия)](#si3)
        - [Проведение вычислений.  $Z$-критерий.](#si4)
        - [Выработка решений на основании полученных результатов](#si5)
    - [Определение объема трафика](#v)
        - [Ошибки первого и второго рода](#m12)
        - [Формула для расчета объема трафика для каждой версии сайта](#fv)
    - [Задание](#t2)
    - [ Приложения](#pr)
        - [ p-value](#pr1)
        - [Критерий Стьюдента](#pr2)


In [ ]:
import pandas as pd
import numpy as np
import random
import scipy as sp
import scipy.stats as sts
import matplotlib.pyplot as plt
%matplotlib inline

random.seed(42)

<a id='rv'></a>
# Случайные величины. Нормальный закон распределения.

Случайные величины бывают:
- дискретные
- непрерывные

На практике статистические заключения опираются на допущения о том, как распределены рассматриваемые случайные переменные (данные). (Даже принято преобразовывать данные с тем, чтобы они соответствовали одному из известных распределений)

<a id='pd'></a>
## Распределение вероятностей

*Закон распределения вероятностей* — это выражение, которое определяет, какие значения будет принимать данная переменная или параметр, и как часто будет встречаться каждое из этих значений.

Иногда закон распределения случайной величины можно выписать сразу. Например, если случайная величина $X$ - число выпадающих  очков при бросках игральной кости (шестигранный кубик с точками), то закон распределения ее значений такой:

| | | | | | | |
|-------------:|:---------|:--------|:---------|:---------|:--------|:---------|
|$$X$$         |1         |2        |3         |4         |5        |6         |
|$$p(X)$$      |1/6       |1/6      |1/6       |1/6       |1/6      |1/6       |


Но далеко не всегда мы  знаем заранее закон распределения случайной величины (ее функцию вероятности). В таком случае мы ее стараемся оценить на основании имеющихся данных. 

Раздобудем результаты $n=200$ бросков игральной кости. Чтобы не бросать кубик самим, смоделируем результаты в Python:

In [ ]:
n=20000
sample = np.random.choice([1,2,3,4,5,6], n)
print("Выпавшие значения Х:",sample )

Тогда вероятности значений случайной величины  X можно оценить с помощью *относительных частот* выпадения значений. 

In [ ]:
# посчитаем число выпадений каждой из сторон:
from collections import Counter

c = Counter()
for i in sample:
    c[i] += 1

print("Число выпадений каждого значения:")    
print(c, '\n')

# получим вероятности, поделив каждое число выпадений на общее количество бросков:

print("Вероятности выпадений каждого значения:")
p_X={key: value/n for key, value in c.items()}
print(p_X)

In [ ]:
# Нарисуем распределение вероятностей:
for key in p_X:
    plt.scatter([key], p_X[key], label=key)
plt.show()

**Задание.** Проведите эксперименты для $n=2000,20000,200000$ бросков. Что происходит с вероятностями? Что это за значение, к которому вероятности стремятся?

<a id='cdf'></a>
## Функция распределения

Знание распределения вероятностей позволяет построить *функцию распределения* $F(x)$. (Обратное тоже верно)))

$F(x)$ - это вероятность того, что случайная величина  $X$  примет значение меньшее, чем $x$, т.е.

$$F(x)=P(X<x)$$

Из этого определения следует очень важная для практики формула вероятности попадания случайной величины $X$ в интервал:

**$$P(a<=X<b)=F(b)-F(a)$$**



In [ ]:
# Построим функцию распределения для нашей выборки. 
from statsmodels.distributions.empirical_distribution import ECDF
F = ECDF(sample,side='left')
plt.step(F.x, F.y, label='F(x)')

plt.ylabel('$F(x)$')
plt.xlabel('$x$')
plt.legend(loc='upper left')

In [ ]:
F([2])

**Задание.**
1. Найдите F(1), F(3), F(6), F(10). Почему получились именно такие значения?

2. Найдите вероятность того, что при броске выпадет не менее 3 и не более 5 очков.

<a id='nb'></a>
## Нормальный закон распределения

В теории статистики доказаны две теоремы, которые обосновывают все статистические вычисления. В силу своей важности эти теоремы получили громкие названия - Центральная предельная теорема и Закон больших чисел.



<a id='lbn'></a>
### Закон больших чисел и центральная предельная теорема

*Закон больших чисел*  утверждает, что какое бы ни было распределение случайной величины, среднее арифметическое ее реализаций при их большом числе близко к истинному значению среднего значения распределения. 

*Центральная предельная теорема* утверждает, что если случайная величина формирует свои значения под действием многочисленных, независимых факторов, влияние каждого из которых пренебрежимо мало, то такая величина распределена нормально. 

То есть имеет распределение, которое называют *нормальным*, и свойства которого очень хорошо изучены. 

Давайте посмотрим на эту теорему в действии. 

*Пример 1.*

Сформируем случайную величину, которая представляет собой сумму $k$ случайных чисел из какого-нибудь распределения (например, равномерного). Сгенерируем  $N$ таких сумм и посмотрим, как они будут распределены.

In [ ]:
N=10000
k=10

In [ ]:
# Убедимся, что умеем генерировать массив из N случайных равномерно распределенных на [0;1] чисел
pd.Series(np.random.random(N)).plot.hist()

In [ ]:
# Складываем покоординатно k массивов, получаем массив из N сумм, строим  гистограмму
pd.Series(sum(np.random.random(N) for _ in range(k))).plot.hist(bins=100)

Узнаем "колокол" нормального распределения.

*Пример 2.* Распределения роста, веса особей популяции - нормальны

In [ ]:
# Длина мышей-белозубок
from IPython.display import Image
Image(url='http://images.vfl.ru/ii/1523220232/73ca30c3/21301892.jpg')

<a id='pdf'></a>
### Формулы для функции и плотности нормального распределения

*Теория.  Взглянуть мельком.*

Функция нормального распределения:

$$Ф(x)=\frac{1}{\sigma\sqrt{2\pi}}\int\limits_{-\infty}^x\exp\left(-\frac{(t-\mu)^2}{2\sigma^2}\right) dt. $$

Плотность нормального распределения

 $$  f(x) = \frac{1}{\sigma\sqrt{2\pi}}\exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right). $$

<a id='gr'></a>
### Генерирование выборки, графики функций распределения и плотностей, эмпирических и теоретических

In [ ]:
# Сгенерировать выборку из нормального распределения можно с помощью пакета scipy.stats
mu=0
sigma=1
n=30 # объем выборки, берите n>10, для работы дальнейших примеров
norm_random_variates = sts.norm(mu, sigma)
sample_1 = norm_random_variates.rvs(n)

In [ ]:
# 10 первых значений
sample_1[:10]

In [ ]:
# Можно и с помощью numpy
sample_2 = np.random.normal(mu,sigma,n)
# 10 первых значений
sample_2[:10]

In [ ]:
# Нарисуем функцию распределения: теоретическую и эмпирическую, составленную по выборке

# теоретическая функция распределения cdf - cumulative density function
x = np.linspace(-5,5,100)
cdf = norm_random_variates.cdf(x)
plt.plot(x, cdf, label='теоретическая Ф(x) ')

# 'эмпирическая функция распределения
F = ECDF(sample_1)
plt.step(F.x,F.y, label='эмпирическая, F(x)')

plt.ylabel('$Ф(x)$')
plt.xlabel('$x$')
plt.legend(loc='upper left')



In [ ]:
# Построим гистограмму
plt.hist(sample_1, bins=6)
plt.ylabel('частота встречаемости значений')
plt.xlabel('$x$')

*Плотностью распределения* называется производная от функции распределения: $f(x)=F'(x)$

In [ ]:
# Нарисуем плотность распределения: теоретическую и эмпирическую, составленную по выборке

# эмпирическая, построенная по выборке с помощью ядерного сглаживания - KDE, Kernel Density Estimation 
# используем библиотеку Pandas:
df = pd.DataFrame(sample_1, columns=['KDE'])
ax = df.plot(kind='density')

# теоретическая плотность - pdf probability density function
x = np.linspace(-5,5,100)
pdf = norm_random_variates.pdf(x)
plt.plot(x, pdf, label='теоретическая плотность распределения', alpha=0.5)
plt.legend()
plt.ylabel('$f(x)$')
plt.xlabel('$x$')

<a id='ci'></a>
## Нахождение доверительных интервалов

In [ ]:
Предварительно сделаем несколько упражнений

*Упр. 1*

Найдем, левее какого числа находится 95% значений нормально распределенной случайной величины с $\mu=0$ и $\sigma=1$, т.е. 95-ю квантиль

In [ ]:
mu=0
sigma=1
sts.norm(mu, sigma).ppf(0.95)

*Упр. 2*

Выполним обратную операцию, убедимся, что вероятность принять значения, меньшие чем 1.6448536269514722, равна 95%


In [ ]:
sts.norm(mu, sigma).cdf(1.644853626951472)

*Упр. 3*

Пользуясь формулой $P(a\le X<b)=F(b)-F(a)$ определим вероятность того, что нормально распределенная случайная величина с $\mu=0$  $\sigma=1$ примет значения $x \in [2;3]$ 

In [ ]:
P=sts.norm(mu, sigma).cdf(3)-sts.norm(mu, sigma).cdf(2)
print(P)

На практике приходится решать и обратную задачу - находить, в каких границах  могут находиться с заданной вероятностью ("надежностью") значения случайной величины.

**Доверительным** называют интервал, который покрывает неизвестный параметр с заданной надёжностью.

В дальнейшем нам нужно будет находить доверительный интервал для математического ожидания  нормально распределенной случайной величины с надежностью $\gamma$. 

*Теория. *


В теории при небольших объемах данных такой интервал можно найти из формулы:
$${P}\left( {\bar X}-t_{\frac{1+\gamma}{2},n-1}\cdot \frac{S}{\sqrt{n}}\le \mu\le {\bar X}+t_{\frac{1+\gamma}{2},n-1}\cdot \frac{S}{\sqrt{n}}\right)=\gamma, $$
где $t_{\alpha,n-1}$ -- $\alpha$-квантили распределения Стьюдента, т.е. распределения случайой величины $T=\frac{{\bar X}-\mu}{S\,/\,\sqrt{n}}$, а $S$ -- несмещеное выборочное стандартное отклонение.

При больших объемах и/или известном стандартном отклонении математического ожидания вместо распределения Стьюдента используется нормальное распределение, и

$${P}\left( {\bar X}-z_{\frac{1+\gamma}{2}}\cdot \frac{\sigma}{\sqrt{n}}\le \mu\le {\bar X}+z_{\frac{1+\gamma}{2}}\cdot \frac{\sigma}{\sqrt{n}}\right)=\gamma, $$
где $z_{\alpha}$ -- $\alpha$-квантили стандартного нормального распределения, т.е. распределения случайной величины $Z=\frac{{\bar X}-\mu}{\sigma\,/\,\sqrt{n}}$, а $S$ -- известное стандартное отклонение.

In [ ]:
#Зададим выборку и надежность 
sample=sample_1
gamma=0.95

Найдем доверительные интервалы для математического ожидания с помощью Python.



In [ ]:
# Реализация вышеприведенной формулы на основе распредеения Стьюдента, применяется
# в случае небольших выборок ($n$<=30), с неизвестным стандартным отклонением, распределение не обязательно нормально

n = len(sample)
m, se = np.mean(sample), sts.sem(sample)
h = se * sts.t._ppf((1+gamma)/2., n-1)
print(m, m-h, m+h)

In [ ]:
# Готовая функция, применимость как примером выше
sts.t.interval(gamma, len(sample)-1, loc=np.mean(sample), scale=sts.sem(sample))

In [ ]:
# Для больших выборок и/или известном стандартном отклонении математического ожидания на основе нормального распределения:
sts.norm.interval(gamma, loc=np.mean(sample), scale=sts.sem(sample))

<a id='t1'></a>
## Задание

Сгенерируйте выборку объемом $n=40$ из нормального распределения с параметрами  $\mu=5$ и $\sigma=2$

Постройте для  выборки :
- гистограмму распределения значений
- теоретическую и эмпирическую функции плотности распределения

Определите интервалы для математического ожидания с надежностью 0.68; 0.95; 0.99 и сравните их с числами 

 $\mu - \sigma$,  $\mu + \sigma$
 
 $\mu - 2\sigma$,  $\mu + 2\sigma$
 
 $\mu - 3\sigma$,  $\mu + 3\sigma$


<a id='ab'></a>
##  A/B тестирование

A/B-тестирование (англ. A/B testing, Split testing) — метод маркетингового исследования, суть которого заключается в том, что контрольная группа элементов сравнивается с набором тестовых групп, в которых один или несколько показателей были изменены, для того, чтобы выяснить, какие из изменений улучшают целевой показатель и улучшают ли.

Типичное применение в веб-дизайне — исследование влияния цветовой схемы, расположения и размера элементов интерфейса на конверсию сайта.

Конверсия (Conversion Rate) в интернет-маркетинге — это отношение числа посетителей сайта, выполнивших на нём какие-либо целевые действия (покупку, регистрацию, подписку, посещение определённой страницы сайта, переход по рекламной ссылке), к общему числу посетителей сайта, выраженное в процентах. 



В теории принципы A/B тестирования невероятно просты:

- Выдвигаем предположение о том, что какое-то изменение (например, персонализация главной страницы) увеличит конверсию интернет-магазина.

- Создаем альтернативную версию сайта «Б» — копию исходной версии «А» с изменениями, от которых мы ждем роста эффективности сайта.

- Всех посетителей сайта случайным образом делим на две равные группы: одной группе показываем исходный вариант (контрольная группа) , второй группе (тестовой) — альтернативный. Одновременно измеряем конверсию для обеих версий сайта.

- Определяем статистически достоверно победивший вариант.

In [ ]:
Image(url='https://d5bygqdtbohob.cloudfront.net/wp-content/themes/vwo/images/ab-test-guide/ab-test.webp')

Мы будем анализировать результаты A/B тестирования двух версий дизайна кнопки сайта интернет-магазина. 

Целевым действием считаем клик по этой кнопке. 

Первые три этапа А/В тестирования за нас провели, результаты предоставили в виде файла ab_dataset.csv. Нам осталось выполнить четвертый пункт.

 <a id='eda'></a>
### Первичный анализ данных

Прочитаем данные из файла `ab_dataset.csv`. Сохраним их в датафрейм `df`. 

Прочитаем данные и посмотрим на первые 5 строк:

In [ ]:
df = pd.read_csv('ab_dataset.csv')

df.head()

Посмотрим, сколько посетителей заходио на сайт (количество строк в нашем датафрейме)

In [ ]:
#количество посетителей всего

n_rows = df.shape[0]
print("Число строк: {}".format(n_rows))

Сколько уникальных пользователей (уникальных `user_id`) в датасете?

In [ ]:
user_total = df.nunique()['user_id']
print("Число уникальных пользователей : {}".format(user_total))

Посетителей из контрольной `control` группы должны были направлять на страницу в старом дизайне  `old_page` , пользователей из тестовой группы `treatment` - на страницу в новом дизайне `new_page`. Проверим, были ли ошибки при направлении.

In [ ]:
mismatch_1 = df.query("group == 'treatment' and landing_page == 'old_page'")
print("Из тестовой группы неверно направлены {}".format(len(mismatch_1)) + " пользователей")

mismatch_2 = df.query("group == 'control' and landing_page == 'new_page'")
print("Из контрольной группы неверно направлены  {}".format(len(mismatch_2)) + " пользователей")


Выясним, есть ли в данных пропуски.

In [ ]:
df.info()

Из сообщений следует, что пропущенных значений нет.

Конверсия по всем посетителям

In [ ]:
p_all=df['converted'].mean()
print("Конверсия по всем посетителям: {} %".format(p_all*100))

In [ ]:
# можно и так:
sum(df['converted'].values)/n_rows

Давайте посмотрим на описательную статистику нашего датасета (воспользуемся функцией  `describe`) и постараемся получить ответы на следующие вопросы:

- Какова вероятность клика для посетителей из контрольной группы (старый дизайн)?

- Какова вероятность клика для посетителей из тестовой группы (новый дизайн кнопки)?

- Каково соотношение размеров тестовой и контрольный групп? Какова вероятность, что очередной посетитель будет направлен на версию со старым дизайном? С новым дизайном?

In [ ]:
df_grp = df.groupby('group')
df_grp.describe()

Ответы на остальные вопросы можно найти, например, так:

In [ ]:
#объем тестовой группы
n_rows_treat = len(df[df['group'] == 'treatment'])

#объем контрольной группы
n_rows_contr = n_rows-n_rows_treat

print("Соотношение размеров тестовой и контрольной групп: {}".format(n_rows_treat/n_rows_contr))

print("Вероятность, что новый пользователь будет направлен на версию со старым дизайном: {}".format(n_rows_treat/n_rows))
print("Вероятность, что новый пользователь будет направлен на версию с новым дизайном: {}".format(n_rows_contr/n_rows))


**Задание **

А теперь ответьте на главный вопрос данного этапа:  выявил ли предварительный анализ, что дизайн кнопки влияет на конверсию и если да, то как именно?

<a id='si'></a>
### Статистический вывод для A/B Теста

По нашим данным очевидно, что кликабельность старой версии выше, чем новой, и напрашивается решение оставить всё как есть.  Однако бизнес потребует обоснований для такого глобального решения, бОльших, чем "очевидно по результатам одного эксперимента". 

И такие обоснования предоставляет специально разработанная методология - статистический вывод. 

Статистический вывод - это переход от данных о статистической выборке (нашего датасета) к обобщениям в виде параметров генеральной совокупности с вычислением степени уверенности в справедливости этих обобщений.

Будем считать, что клик — это некоторая случайная переменная , принимающая значения 1  или 0 с вероятностями $\theta$ и $1-\theta$ соответственно. 

Применительно к нашей задаче посетитель может кликнуть на кнопку (с вероятностью $\theta$) или не кликнуть на нее (с вероятностью, соответственно,  $1-\theta$)

На практике статистические заключения опираются на допущения о том, как распределены рассматриваемые случайные переменные (данные). (Даже принято преобразовывать данные с тем, чтобы они соответствовали одному из известных распределений)

Распределение вероятностей — это выражение, которое определяет, какие значения будет принимать данная переменная или параметр,  и как часто будет встречаться каждое из этих значений.

 Наша случайная переменная — клик — имеет распределение, которое называется распределением Бернулли с параметром $\theta$:
$$ p(k) = \theta^k (1 - \theta)^{1 - k}, $$

где $p(k)$ —  вероятность случайной переменной  принять значение $k$, $k \in \{ 0;1\}$.

Из теории известно, что среднее значение распределения Бернулли равно $\mu = \theta$, а дисперсия равна $\sigma^2 = \theta(1 - \theta)$. Нас интересует конверсия сайта, в рассматриваемой постановке она равна $\theta$.

Основу статистического вывода составляет проверка гипотез. Проверка гипотез состоит из основных этапов:

- Формулировка рабочей гипотезы, которая может быть проверена статистическими методами. 

- Формальное описание нулевой и альтернативной гипотез. 


*Нулевая гипотеза* - это гипотеза об отсутствии различий. Она обозначается $H_0$ и содержит 0 в обозначении.


*Альтернативная гипотеза* - это гипотеза о значимости различий, о том, что различия не случайны, о том, что мы хотим подтвердить, когда хотим доказать значимость различий. Обозначается $H_1$.

- Выбор подходящего статистического теста (статистического критерия), сбор данных, проведение вычислений. 
Don't worry!! В руководствах и справочниках при описании каждого критерия даются как формулировки гипотез, которые он помогает нам проверить, так и условия применимости. 

- Выработка решений на основании полученных результатов.

<a id='si1'></a>
#### Формулировка рабочей гипотезы

Рабочая гипотеза в нашем случае может быть сформулирована, например, так: "Конверсия сайта со старым дизайном не меньше, чем с новым"

<a id='si2'></a>
#### Формальное описание нулевой и альтернативной гипотез

$H_{0}$ : $\theta_{new}$ <= $\theta_{old}$ 

$H_{1}$ : $\theta_{new}$ > $\theta_{old}$ 

<a id='si1'></a>
#### Выбор подходящего статистического теста (статистического критерия)

Истинные значения $\theta_{new}$ и $\theta_{old}$ мы найти не можем, но мы можем их сравнить. В этом помогут те самые две теоремы с громкими названиями.






На основании Закона больших чисел мы можем оценить средние значения случайных величин $\theta_{new}$ и $\theta_{old}$ на  генеральных совокупностях по их средним значениям на наших больших выборках.

На основании Центральной предельной теоремы средние значения распределены нормально при больших объемах выборок.

Чтобы выбрать для проверки наших гипотез подходящий статистический критерий, уясним задачу. Нам нужно выяснить, случайно или значимо отличаются средние значения, т.е. доли, кликнувших посетителей в контрольной и тестовой выборках.

Вообще-то доли относятся к категориальным данным. В разделе о проверке гипотез для категориальных данных увидим, что пользоваться можно и точным критерием Фишера, и критерием хи-квадрат.

Однако в случае достаточно большого объема данных в силу центральной предельной теоремы распределение средних, которым мы оцениваем $\theta_{new}$ и $\theta_{old}$, становится похожим на нормальное распределение. Значит,  можно пользоваться критериями для сравнения средних значений для количественных случайных величин, например, $t$-критерием, или использовать $Z$-критерий.

Им и воспользуемся.

Эмпирическое правило применимости $Z$-критерия $n \cdot \theta \ge 5$, $n \cdot (1-\theta) \ge 5$ в нашем случае выполнено  (проверьте!).

 Согласно описанию критерия, нужно по данным посчитать значение $Z$-статистики, сравнить его с критическим значением $Z$-статистики при задаваемом нами уровне значимости $\alpha$, и, если первое больше, то гипотеза $H_0$ отвергается с уровнем значимости $\alpha$, если меньше — принимается. 

Уровень значимости $\alpha$ — это вероятность того, что, когда мы сочли различия существенными, они на самом деле случайны. 



<a id='si4'></a>
#### Проведение вычислений. $Z$-критерий.

In [ ]:
#Z-статистика
import statsmodels.api as sm

convert_contr = sum(df.query("group == 'control'")['converted'])
convert_treat = sum(df.query("group == 'treatment'")['converted'])

z_score, p_value = sm.stats.proportions_ztest([convert_contr, convert_treat], [n_rows_contr, n_rows_treat], alternative='smaller')
print("Z-статистика={},  p_value={}".format(z_score, p_value))


Мы нашли значение $Z$-статистики:  z_score = 1.31092419842

Одновременно выдалось значение $p-value$ = 0.905058312759

$p-value$ - это фактический уровень значимости, он вычисляется как вероятность получения данных или меньших различий контрольной и тестовой групп в случае, если гипотеза $H_0$ верна. 

Найдем крититическое значение $Z$-статистики при  уровне значимости $\alpha=0.05$

In [ ]:
from scipy.stats import norm
alpha=0.05
print(norm.cdf(z_score))

# ppf -  percent point function, квантильная функция, обратная к cdf 
print("Критическое значение при {} уровне значимости: {}".format(alpha, norm.ppf(1-(alpha))))


<a id='si5'></a>
#### Выработка решений на основании полученных результатов

Мы видим, что значение $Z$-статистики меньше   критического значения $Z$-статистики при уровне значимости $\alpha=0.05$, значит,  нет оснований отвергать $H_0$, а, следовательно, нет оснований отказываться от старой кнопки.

Это и есть основной вывод проведенного  А/В теста.

<a id='v'></a>
## Определение объема трафика

(*Изложение следует тексту статьи * https://habrahabr.ru/company/ods/blog/325416/  "Байесовские многорукие бандиты против A/B тестов")

Остается один важный вопрос: а какое количество трафика (пользователей) пустить на каждую вариацию сайта? 

Формула для расчета нужного объема данных есть. Но чтобы ею воспоьзоваться, придется  встретиться с отделом маркетинга и задать им несколько вопросов, и, вероятно, придется им объяснить, что такое ошибки первого и второго рода. Посмотрим, что это за ошибки, и определим вопросы, которые следует задать отделу маркетинга, чтобы оценить количество трафика на вариацию. В конечном счете именно маркетинг принимает окончательно решение о том, какой цвет кнопки оставить.

<a id='m12'></a>
### Ошибки первого и второго рода

|  | | | |
|----------|:---------|:--------|:---------|
|  | | 	Верная гипотеза: | |
|  |     | $H_0$   | $H_1$    |
| Результат применения критерия: | $H_0$   |$H_0$ верно принята  |$H_0$ неверно принята (ошибка II рода)  |  
|  |$H_1$   |$H_0$ неверно отвергнута(ошибка I рода)  | $H_0$ верно отвергнута   |



Уровень значимости (статистическая значимость, statistical significance) $\alpha$ - это и есть вероятность ошибки первого рода, т. е. вероятность принятия альтернативной гипотезы при условии, что на самом деле верна нулевая гипотеза. 

Обозначим  $\beta$  вероятность ошибки второго рода. 

Величина  $1-\beta$ называется *статистической мощностью* (statistical power) критерия. По сути мощность показывает, сколько значений, соответствующих альтернативной гипотезе, мы действительно отнесем к альтернативной гипотезе

<a id='fv'></a>
### Формула для расчета объема трафика для каждой версии сайта

 $$ 
 n = \left(\frac{t_{\beta}\sqrt{\theta_{test}\left(1 - \theta_{test}\right)} - t_{1-\alpha}\sqrt{\theta_{control}\left(1 - \theta_{control}\right)}}{\theta_{control} - \theta_{test}}\right)^2 
 $$

Таким образом, чтобы вычислить эффективный размер выборки, мы должны пойти в маркетинг или другой бизнес-департамент и узнать у них следующее:

- какое значение конверсии $\theta_{test}$ на новой вариации должно получиться, чтобы бизнес принял решение перейти со старой версии кнопки с конверсией $\theta_{control}$  на новую; естественно предположить, что если новое больше старого, то уже достаточно оснований для внедрения, но не стоит забывать про накладные расходы, как минимум на тестирование, ведь чем меньше разница, тем больше трафика нужно потратить на тест, а это потеря прибыли (при этом существует риск, что тест провалится);
- какой допустимый уровень значимости  и мощности  необходим (вероятно, тут еще придется как то объяснить бизнесу, что это значит).

In [ ]:
alpha=0.05
beta=0.1
theta_c=df.query("group == 'control'")['converted'].mean()
theta_t=theta_c+0.01

In [ ]:
print(theta_c,theta_t)

In [ ]:
#Реализуем функцию вычисления объема траффика
def get_size(theta_c, theta_t, alpha, beta):
    # вычисляем квантили нормального распределения
    t_alpha = sts.norm.ppf(1 - alpha, loc=0, scale=1)
    t_beta = sts.norm.ppf(beta, loc=0, scale=1)
    # решаем уравнение относительно n
    n = t_alpha*np.sqrt(theta_t*(1 - theta_t))
    n -= t_beta*np.sqrt(theta_c*(1 - theta_c))
    n /= theta_c - theta_t
    return int(np.ceil(n*n))

n_max = get_size(theta_c,theta_t,alpha,beta)
print ("Необходимый трафик для каждой версии сайта: {}".format(n_max))
# выводим порог, выше которого отклоняется H_0
print ("Пороговое значение конверсии тестового сайта, выше которого отклоняется H0:")
print  (theta_c + sts.norm.ppf(1 - alpha, loc=0, scale=1)*np.sqrt(theta_c*(1 -  theta_c)/n_max))

<a id='t2'></a>
## Задание

Тестирование двух версий сайта выявило следующие показатели:

|Версия сайта  |Объем трафика |Количество кликов | Выборочное математическое ожидание конверсии|Выборочное среднеквадратическое отклонение конверсии
|:----------|:---------|:--------|:---------|:---------|
| Контрольная |256000 | 28160 | ? | 0.34|
| Тестовая |256000     | 58880   |?   |0.37|

- Заполнить пропущенные значения в таблице
- Сформулировать нулевую и альтернативную гипотезы
- Проверить, являются ли различия значимыми при уровне значимости $\alpha=0.95$


<a id='pr'></a>
## Приложения

<a id='pr1'></a>
### p-value

In [ ]:
# ------- ДОПОЛНИТЕЛЬНЫЙ МАТЕРИАЛ ----- p-value---------
# Мы можем явно оценить p-value с помощью моделирования. 
# Пусть верна H0, конверсии обеих версий одинаковы и равны p_all (p_all=df['converted'].mean())
# Проведем не одно А/В тестирование, а 10000 тестирований с теми же трафиками.
# Определим долю тестирований, в которых различия такие же или меньшие, чем в данном датасете
p_diffs = []
p_treat=p_all
p_contr=p_all

for _ in range(10000):
    new_page_converted = np.random.choice([1, 0], size=n_rows_treat, p=[p_treat, (1-p_treat)]).mean()
    old_page_converted = np.random.choice([1, 0], size=n_rows_contr, p=[p_contr, (1-p_contr)]).mean()
    diff = new_page_converted - old_page_converted 
    p_diffs.append(diff)
# вычислим разницу конверсий в данных ab_data.csv.
act_diff = df[df['group'] == 'treatment']['converted'].mean() -  df[df['group'] == 'control']['converted'].mean()
# Она отрицательна
# доля различий p_diffs, больших чем наблюдаемые в ab_data.csv:
np.mean(list((act_diff < x for x in p_diffs)))

<a id='pr2'></a>
### Критерий Стьюдента

In [ ]:
#------- -ДОПОЛНИТЕЛЬНЫЙ МАТЕРИАЛ ---- МАЛЫЕ ВЫБОРКИ ----
# Использование t-критерия для сравнения долей
import statsmodels
print("t_score, p-value, df:")
statsmodels.stats.weightstats.ttest_ind(df.query("group == 'control'")['converted'], \
                                df.query("group == 'treatment'")['converted'], alternative="smaller")

In [ ]:
print("Критическое значение t-статистики={}".format(sts.t.ppf(q=1-(alpha), df= 290582)) )